1) Проанализируйте переменные (столбцы) и укажите, для каких из них определение
числовых характеристик имеет смысл. Понимать значения всех характеристик, которые дает метод describe().

In [ ]:
"Date, Seasons, Holiday, Functioning Day, Temperature Category"

2) Для каждого сезона найдите: среднюю, медианную и модовую температуры, дисперсию, среднеквадратичное отклонение, размах и интерквартильный размах
температуры. Результат также выведите в виде общей таблицы.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

bikes_df = pd.read_csv("DataBikes_new.csv", sep=',')
bikes_df.columns = bikes_df.columns.str.strip()

seasons_groupby = bikes_df.groupby('Seasons')['Temperature']

bikes_num_desc_df= pd.DataFrame(
    {
        "mean": seasons_groupby.mean(),
        "median": seasons_groupby.median(),
        "mode": seasons_groupby.apply(
            lambda x: x.mode().iloc[0] if not x.mode().empty else None
        ),
        "variance": seasons_groupby.var(),
        "std": seasons_groupby.std().round(3),
        "range": seasons_groupby.max() - seasons_groupby.min(),
        "IQR": seasons_groupby.quantile(0.75) - seasons_groupby.quantile(0.25),
    }
)

display(bikes_num_desc_df)
seasons_groupby.describe()

,mean,median,mode,variance,std dev,range,IQR
Seasons,,,,,,,
Autumn,14.138821,13.8,10.8,49.589573,7.042,33.5,11.1
Spring,13.038533,13.3,9.6,43.665630,6.608,36.0,9.9
Summer,26.574827,26.6,23.4,21.655814,4.654,23.1,6.7
Winter,-2.567783,-2.2,-1.8,29.877730,5.466,28.1,7.7


,count,mean,std,min,25%,50%,75%,max
Seasons,,,,,,,,
Autumn,2138.0,14.138821,7.041986,-3.0,8.7,13.8,19.8,30.5
Spring,2154.0,13.038533,6.607997,-6.6,8.2,13.3,18.1,29.4
Summer,2169.0,26.574827,4.653581,16.3,23.1,26.6,29.8,39.4
Winter,2120.0,-2.567783,5.466052,-17.8,-6.2,-2.2,1.5,10.3


3) Переменная «Temperature» содержит None значения. Предложите и реализуйте способ их замены, который по Вашему мнению будет наиболее достоверно соответствовать
реальным условиям. Определите, как изменятся при этом числовые характеристики
среднего и разброса для температуры.

In [2]:
bikes_df.groupby('Seasons')['Temperature'].apply(lambda x: x.isna().sum())

Seasons
Autumn    46
Spring    54
Summer    39
Winter    40
Name: Temperature, dtype: int64

In [ ]:
bikes_df['Temperature'] = seasons_groupby.transform(lambda x: x.fillna(x.mean()))

# new_numerical_chars = pd.DataFrame({
#     "mean": seasons_groupby.mean(),
#     "mode": seasons_groupby.apply(lambda x: x.mode().tolist()),
#     "variance": seasons_groupby.var(),
#     "std": seasons_groupby.std().round(3),
#     "IQR": seasons_groupby.quantile(0.75) - seasons_groupby.quantile(0.25)
# })

bikes_df.groupby("Seasons")["Temperature"].describe()
# print(bikes_df['Temperature'].isna().sum())
# print(new_numerical_chars)

,count,mean,std,min,25%,50%,75%,max
Seasons,,,,,,,,
Autumn,2184.0,14.138821,6.967397,-3.0,8.800,14.138821,19.700,30.5
Spring,2208.0,13.038533,6.526656,-6.6,8.300,13.038533,17.900,29.4
Summer,2208.0,26.574827,4.612281,16.3,23.175,26.574827,29.700,39.4
Winter,2160.0,-2.567783,5.415181,-17.8,-6.100,-2.400000,1.425,10.3


4) Предположим, что наблюдения температуры, отклоняющиеся от средней температуры для каждого сезона более чем на два стандартных отклонения, являются выбросами.
Сформируйте таблицу без выбросов. Установить, будет ли наблюдаться отличие в характеристиках средних для температуры, построенных на основе исходной и новой
таблиц.

In [ ]:
mean = seasons_groupby.transform('mean')
std = seasons_groupby.transform('std')

outliers = (bikes_df['Temperature'] - mean).abs() > 2 * std  # нахождение выбросов

no_outliers_df = bikes_df[~outliers].copy()  # копия датафрейма без выбросов
new_seasons_groups = no_outliers_df.groupby('Seasons')['Temperature']

new_numerical_chars = pd.DataFrame({
    "mean": seasons_groupby.mean(),
    "new_mean": new_seasons_groups.mean()
})

print(new_numerical_chars)

5) На одном графике построить размах для каждого месяца. Подписать название графика и его оси.

In [ ]:
bikes_df['Date'] = pd.to_datetime(bikes_df['Date'], format="%d/%m/%Y")
bikes_df['Month'] = bikes_df['Date'].dt.month  # достать месяца в новый столбец

month_range = bikes_df.groupby('Month')['Temperature'].apply(lambda x: x.max() - x.min())
print(month_range)

# график
plt.figure(figsize=(12, 6))
plt.bar(month_range.index, month_range.values)

plt.title("Размах температуры по месяцам")
plt.xlabel("Месяц")
plt.ylabel("Размах температуры (max - min)")
plt.grid(axis="y", linestyle="--", alpha=0.4)

plt.show()

6) Найдите, сколько дней работа проката по оценкам была выше среднего. В этом случае следует учитывать, что оценки выставлялись по разным шкалам, поэтому 10 по одной
шкале это отлично, а по другой очень плохо. Для решения этой проблемы используют так называемую «z-стандартизацию», т. е. вводят новую переменную z = (x - m) / sigma,
которая показывает на сколько среднеквадратичных отклонений, x отклоняется от среднего.

In [ ]:
bikes_df['z_score'] = bikes_df.groupby('score_type')['score'].transform(lambda x: (x - x.mean()) / x.std())

bikes_df['above_mean'] = bikes_df['z_score'] > 0

above_mean_count = bikes_df['above_mean'].sum()

print("Кол-во дней, когда оценка была выше среднего:", above_mean_count)

7) Установить наличие линейной зависимости между температурой и числом арендованных велосипедов (по данным за весь год, по сезонам, по месяцам). Для данных,
имеющих максимальную линейную зависимость, построить график рассеяния.

In [ ]:
corr_year = bikes_df['Temperature'].corr(bikes_df['Rental count'])
print("Зависимость между температурой и числом арендованных велосипедов за весь год: ", corr_year)

corr_seasons = bikes_df.groupby('Seasons').apply(lambda x: x['Temperature'].corr(x['Rental count']))
print("\nЗависимость между температурой и числом арендованных велосипедов по сезонам:\n", corr_seasons)

corr_month = bikes_df.groupby('Month').apply(lambda y: y['Temperature'].corr(y['Rental count']))
print("\nЗависимость между температурой и числом арендованных велосипедов по месяцам:\n", corr_month, '\n')

In [ ]:
all_corr = pd.concat([pd.Series({'year': corr_year}), corr_seasons.rename(lambda s: f"season_{s}"), corr_month.rename(lambda m: f"month_{m}")])
print(all_corr)

best_label = all_corr.idxmax()
best_value = all_corr.loc[best_label]
print(best_label)

print("\nМаксимальная зависимость:", best_label, "=", best_value)

In [ ]:
if best_label == "year":
    plot_df = bikes_df
elif "season_" in best_label:
    season = best_label.replace("season_", "")
    plot_df = bikes_df[bikes_df["Seasons"] == season]
elif "month_" in best_label:
    month = int(best_label.replace("month_", ""))
    plot_df = bikes_df[bikes_df["Month"] == month]

plot_df = plot_df.sort_values("Temperature")

plt.plot(plot_df["Temperature"], plot_df["Rental count"])
plt.title(f"Зависимость температуры и аренд ({best_label})")
plt.xlabel("Температура")
plt.ylabel("Аренды")
plt.grid(True)
plt.show()

8) Аналогичные п.7 исследования провести для температуры и оценки.

In [ ]:
corr_year_score = bikes_df["Temperature"].corr(bikes_df["score"])
print("Зависимость между температурой и оценкой за весь год:", corr_year_score)

corr_seasons_score = bikes_df.groupby("Seasons").apply(lambda x: x["Temperature"].corr(x["score"]))
print("\nЗависимость между температурой и оценкой по сезонам:", corr_seasons_score)

corr_month_score = bikes_df.groupby('Month').apply(lambda y: y['Temperature'].corr(y['score']))
print("\nЗависимость между температурой и числом арендованных велосипедов по месяцам:", corr_month_score, '\n')

In [ ]:
all_corr_score = pd.concat([pd.Series({'year': corr_year_score}), corr_seasons_score.rename(lambda s: f"season_{s}"), corr_month_score.rename(lambda m: f"month_{m}")])
print(all_corr_score)

best_label_score = all_corr_score.abs().idxmax()
best_value_score = all_corr_score.loc[best_label_score]

print("\nМаксимальная зависимость:", best_label_score, "=", best_value_score)

In [ ]:
if best_label_score == "year":
    plot_df = bikes_df
elif "season_" in best_label_score:
    s = best_label_score.replace("season_", "")
    plot_df = bikes_df[bikes_df["Seasons"] == s]
elif "month_" in best_label_score:
    m = int(best_label_score.replace("month_", ""))
    plot_df = bikes_df[bikes_df["Month"] == m]


plot_df = plot_df.sort_values("Temperature")

plt.plot(plot_df["Temperature"], plot_df["score"])
plt.title(f"Зависимость температуры и оценки ({best_label_score})")
plt.xlabel("Температура")
plt.ylabel("Оценка")
plt.grid(True)
plt.show()
